# Data Detective Defective

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from random import sample
import numpy as np

# Define a class to manage the game logic
class InteractiveGuessingGame:
    def __init__(self):
        # List of items found in a classroom
        self.objects = [
            "chalkboard or whiteboard",
            "desks and chairs",
            "textbooks",
            "notebooks or writing pads",
            "pens and pencils",
            "projector",
            "computer",
            "clock",
            "bulletin board",
            "maps or globes"
        ]
        
        # List of probabilities for each item being in a classroom
        self.probabilities = [
            0.02, 0.05, 0.10, 0.15, 0.20,
            0.02, 0.05, 0.10, 0.15, 0.20
        ]

        self.payout_multipliers = [20, 10, 8, 5, 3, 20, 10, 8, 5, 3]
        self.max_attempts = 5
        self.max_wager = 5.00  # Max wager
        self.max_payout = 500  # Max payout per game
        self.discount = 0.96  # Discount for security
        self.reset_game()

    def reset_game(self):
        self.secret_objects = sample(self.objects, self.max_attempts)
        self.secret_probabilities = [self.probabilities[self.objects.index(obj)] for obj in self.secret_objects]
        self.guess_history = []
        self.correct_guesses = []
        self.incorrect_guesses = []
        self.other_guesses = []
        self.attempts = 0

    def check_guess(self, guess):
        self.attempts += 1
        self.guess_history.append(guess.lower())
        if guess.lower() in [obj.lower() for obj in self.secret_objects]:
            self.correct_guesses.append(guess.lower())
            return True
        elif guess.lower() in self.objects:
            self.incorrect_guesses.append(guess.lower())
        else:
            self.other_guesses.append(guess.lower())
        return False

    def get_results(self):
        correct = set(self.correct_guesses)
        incorrect = set(self.incorrect_guesses) - correct
        other = set(self.other_guesses) - correct - incorrect
        return correct, incorrect, other

    def calculate_payoff(self, wager):
        payoff = 0
        details = []
        for guess in self.correct_guesses:
            index = self.objects.index(guess)
            prob = self.probabilities[index]
            payout_multiplier = self.payout_multipliers[index]
            amount = min(payout_multiplier * wager * self.discount, self.max_payout)
            payoff += amount
            details.append((guess, prob, payout_multiplier, amount))
        return payoff, details

# Function to manage the interactive widgets and game state
def game_widget():
    game = InteractiveGuessingGame()
    name_input = widgets.Text(
        placeholder='Enter your name here...',
        description='Name:',
    )
    guess_input = widgets.Dropdown(
        options=game.objects,
        description='Guess:'
    )
    wager_input = widgets.Dropdown(
        options=[
            ("$5.00 (max)", 5.00),
            ("$1.00", 1.00),
            ("$0.25", 0.25),
            ("$0.10", 0.10)
        ],
        description='Wager:'
    )
    button = widgets.Button(description="Check Guess")
    retry_button = widgets.Button(description="Retry", disabled=True)
    output = widgets.Output()

    def on_button_clicked(b):
        with output:
            clear_output()
            if name_input.value == "":
                print("Please enter your name first!")
                return
            if wager_input.value > game.max_wager:
                print(f"Wager exceeds the maximum limit of ${game.max_wager}")
                return
            guess = guess_input.value.lower()
            if game.check_guess(guess):
                print(f"Correct, {name_input.value}! The object {guess} is one of the secret objects.")
            else:
                print(f"Incorrect, {name_input.value}. Try again!")
            
            tries_left = game.max_attempts - game.attempts
            if tries_left > 0:
                print(f"You have {tries_left} more tries left.")
            else:
                print(f"No more attempts left, {name_input.value}. The objects were {', '.join(game.secret_objects)}.")
                guess_input.disabled = True
                button.disabled = True
                retry_button.disabled = False
                correct, incorrect, other = game.get_results()
                payoff, details = game.calculate_payoff(wager_input.value)
                print(f"Correct guesses: {correct}")
                print(f"Incorrect but on list: {incorrect}")
                print(f"Other incorrect guesses: {other}")
                print(f"Congratulations! You've won ${payoff:.2f}!")
                print("Payoff details:")
                for guess, prob, multiplier, amount in details:
                    print(f"Guess: {guess}, Probability: {prob}, Multiplier: {multiplier}, Payout: ${amount:.2f}")

    def on_retry_button_clicked(b):
        with output:
            clear_output()
            game.reset_game()
            guess_input.disabled = False
            button.disabled = False
            retry_button.disabled = True
            print("New game started. Make your guesses!")

    button.on_click(on_button_clicked)
    retry_button.on_click(on_retry_button_clicked)
    display(name_input, guess_input, wager_input, button, output, retry_button)

# Display the game widget in the notebook
game_widget()


Text(value='', description='Name:', placeholder='Enter your name here...')

Dropdown(description='Guess:', options=('chalkboard or whiteboard', 'desks and chairs', 'textbooks', 'notebook…

Dropdown(description='Wager:', options=(('$5.00 (max)', 5.0), ('$1.00', 1.0), ('$0.25', 0.25), ('$0.10', 0.1))…

Button(description='Check Guess', style=ButtonStyle())

Output()

Button(description='Retry', disabled=True, style=ButtonStyle())